In [1]:
#requiremetns
#!pip install bs4
#!pip install requests
#!pip install pandas

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [3]:
def get_url(search_item):
    template = 'https://www.flipkart.com/search?q={}&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_1_2_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_1_2_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=mobile+phones&requestId=e4af82af-8275-4679-9d0f-b2dba2af3686&as-searchtext=mo'
    search_item = search_item.replace(" ","+")
    url = template.format(search_item)
    url += '&page={}'
    return url

def extract_phone_model_info(item):
    model = item.find('div',{'class':"_4rR01T"}).text
    ram = item.find('li',{'class':"rgWa7D"}).text[0:item.find('li',{'class':"rgWa7D"}).text.find('|')]
    price = item.find('div',{'class':'_30jeq3 _1_WHN1'}).text
    result = [model,ram,price]
    return result

def main(search_item):
    df = pd.DataFrame(columns=['Model', 'RAM in GB', 'PRICE'])
    url = get_url(search_item)
    for page in range(1,500):
        source=requests.get(url.format(page))
        soup=BeautifulSoup(source.text,'html')
        results = soup.find_all('a',{'class':"_1fQZEK"})
        for item in results:
            result = extract_phone_model_info(item)
            if result[1].find('GB RAM') != -1:
                result[1] = int(result[1].split()[0])
                df.loc[len(df.index)] = result
    return df

In [4]:
df = main('mobile phones')
df.sort_values(by=['RAM in GB','PRICE'],ascending=[False,True],inplace=True)
print("Mobile Phone with Highest RAM and Lowest Price in flipkart is:")
print(df.iloc(0)[0])

Mobile Phone with Highest RAM and Lowest Price in flipkart is:
Model        POCO F4 5G (Night Black, 256 GB)
RAM in GB                                  12
PRICE                                 ₹29,999
Name: 262, dtype: object


In [5]:
#all the mobile phones scraped from flipkart
#sorted according to descending order of RAM and ascending order of PRICE
df

,Model,RAM in GB,PRICE
262,"POCO F4 5G (Night Black, 256 GB)",12,"₹29,999"
581,"vivo V25 5G (Surfing Blue, 256 GB)",12,"₹31,999"
662,"vivo V25 5G (Surfing Blue, 256 GB)",12,"₹31,999"
216,"Nothing Phone (1) (White, 256 GB)",12,"₹36,399"
304,"Nothing Phone (1) (Black, 256 GB)",12,"₹36,399"
...,...,...,...
545,"I Kall K2 (Gold, 8 GB)",1,"₹3,989"
559,"I Kall K2 (Gold, 8 GB)",1,"₹3,989"
595,"I Kall K2 (Gold, 8 GB)",1,"₹3,989"
643,"I Kall K2 (Gold, 8 GB)",1,"₹3,989"
